In [ ]:
import numpy as np
import tensorflow as tf

#Load and Prepare the MNIST Dataset

In [ ]:
# Load the MNIST dataset and split it into training and test sets
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# Convert 0/255 to 0/1
x_train_int = [np.round(1.0 * i / 256) for i in x_train]
x_test_int = [np.round(1.0 * i / 256) for i in x_test]

#Naive Bayes Classifier

##Calcualte $p(x_i=0|y)$ and $p(x_i=1|y)$

###Count the number of pixels equal to 0 and 1 for each training sample

In [ ]:
x_train_flat = np.reshape(x_train_int, [60000, 784])
zeros_cnt = np.zeros([10, 784])
ones_cnt = np.zeros([10, 784])
total_each_class = np.zeros([10, 1])

# Loop over the 10 classes
for i in range(10):
  # Loop over all the training examples
  for j in range(len(x_train_flat)):
    # Count the number of occurances of each class
    if y_train[j] == i:
      total_each_class[i] += 1
      # Loop over every pixel of the training sample
      for k in range(784):
        # Count the number of pixels equal to 1 in the training sample
        if x_train_flat[j, k] == 1:
          ones_cnt[i, k] += 1
# Get the number of pixels equal to 0 in all training samples
zeros_cnt = total_each_class - ones_cnt

###Estimate the log probabilities

In [ ]:
p_x_0 = np.zeros([10, 784])
logp_x_0 = np.zeros([10, 784])

# Loop over the 10 classes
for i in range(10):
  # Loop over all the pixels
  for j in range(784):
    # Check if there are no pixels equal to 0 
    if zeros_cnt[i, j] == 0:
      # Laplace smoothing
      p_x_0[i, j] = (zeros_cnt[i, j] + 0.7) / (total_each_class[i] + 0.7)
    else:
      p_x_0[i, j] = zeros_cnt[i, j] / total_each_class[i]
    # Calculate the log probabilites to avoid multiplying by a small number
    logp_x_0[i, j] = np.log10(p_x_0[i, j])

In [ ]:
p_x_1 = np.zeros([10, 784])
logp_x_1 = np.zeros([10, 784])

# Loop over the 10 classes
for i in range(10):
  # Loop over all the pixels
  for j in range(784):
    # Check if there a no pixels equal to 1
    if ones_cnt[i, j] == 0:
      # Laplce smoothing
      p_x_1[i, j] = (ones_cnt[i, j] + 0.7) / (total_each_class[i] + 0.7)
    else:
      p_x_1[i, j] = ones_cnt[i, j] / total_each_class[i]
    # Calculate the log porbabilities to avoid multiplying by a small number
    logp_x_1[i, j] = np.log10(p_x_1[i, j])

##Naive Bayes classification function

In [ ]:
def naive_bayes(_x_test):
  p_x_test = np.zeros([10, 1])
  # Loop over the 10 classes
  for i in range(10):
    # Loop over all the pixels
    for j in range(784):
      # Check if the current pixel is equal to 0 or 1
      if _x_test[j] == 0:
        p_x_test[i] += logp_x_0[i, j]
      elif _x_test[j] == 1:
        p_x_test[i] += logp_x_1[i, j]
  y = np.argmax(p_x_test)
  return y

##Predict the test classes

In [ ]:
x_test_flat = np.reshape(x_test_int, [10000, 784])
y_hat = []

for i in range(10000):
  y_hat.append(naive_bayes(x_test_flat[i]))

print('The empirical risk when using the missclassification loss function =', np.count_nonzero(y_hat - y_test) / 10000)
print('The empirical risk when using the squared error loss function =', sum((y_hat - y_test)**2) / 10000)

The empirical risk when using the missclassification loss function = 0.1565
The empirical risk when using the squared error loss function = 2.8319
